<a href="https://colab.research.google.com/github/carlm451/LSTM_Character_Language_Model/blob/main/Notebook1_RNN_vs_LSTM_Comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Performance comparison between basic RNN and GPU enabled LSTM

## Training Data: Homer's Illiad 

![picture](https://upload.wikimedia.org/wikipedia/commons/0/0d/Iliad_VIII_245-253_in_cod_F205%2C_Milan%2C_Biblioteca_Ambrosiana%2C_late_5c_or_early_6c.jpg)



In [ ]:
import requests



In [ ]:
# data I/O
data = open('illiad_clean.txt', 'r').read() # should be simple plain text file





#data = 'abcdefghijklmnopqrstuvwxyz0123456789'*1000
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print('data has %d characters, %d unique.' % (data_size, vocab_size))
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

## vanilla RNN min-char-rnn.py by Andrej Karpathy

In [ ]:
import numpy as np

# hyperparameters
hidden_size = 128 # size of hidden layer of neurons
seq_length = 100 # number of steps to unroll the RNN for
learning_rate = 1e-1

# model parameters
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output
bh = np.zeros((hidden_size, 1)) # hidden bias
by = np.zeros((vocab_size, 1)) # output bias

In [4]:
import requests

url = 'https://raw.githubusercontent.com/carlm451/LSTM_Character_Language_Model/main/jax_char_lstm.py?token=GHSAT0AAAAAACCB2Q2HCKE6IPMH4BBMK3KYZCRGQMA'
r = requests.get(url)

with open('jax_char_lstm.py','w') as file:
     file.write(r.text)

In [5]:
import jax_char_lstm as lstm

## LSTM module --- JAX + Adam optimizations + embedding layer 